<h1><center>Redes Neuronales con Keras y Scikit</center></h1>


<center><i>Clasificación binaria: Predicting Red Hat Business Value</i></center>

# <h1>Contenido</h1>

<div class="alert alert-block alert-warning" style="margin-top: 20px">
    <ol>
        <li><a href="#problema">Contexto Analítico</a></li>   
        <li><a href="#ingenieria">Ingeniería de datos</a></li>          
        <li><a href="#arquitectura">Arquitectura propuesta</a></li>
        <li><a href="#evaluaion">Evaluación del Modelo</a></li>
    </ol>
</div>
<br>
<hr>

# <h1 id="problema">Contexto analítico</h1>

Este caso se basa en un reto propuesto por Red Hat en la plataforma Kaggle. (https://www.kaggle.com/c/predicting-red-hat-business-value/data) 

Red Hat es capaz de obtener una importante cantidad de información de comportamiento de sus clientes. El interes de la compañia se basa en la busqueda de mejores metodos para analizar sus clientes para predecir sobre cuales deberian hacer un especial enfasis que podrian convertirse en clientes de alto valor.

En este reto se propone crear un algoritmo de clasificación que identifique los clientes con mayor potencial para convertirse en clientes de alto valor para Red Hat basados en su actividad.

Se cuenta con dos datasets (people.csv y act_train.csv) los cuales cargaremos directamente a Colab.


In [1]:
#Importamos las librerias necesarias
import pandas as pd
import numpy as np
import io
from google.colab import files #Librería necesaria para interactuar con archivos en Colab

In [2]:
#Cargamos los dataset 
# Opcion 1: usando el gestor de archivos grafico
# uploaded = files.upload()
# people = pd.read_csv(io.StringIO(uploaded['people.csv'].decode('utf-8')), sep = ',' )

In [3]:
# Opcion 2 metodo convencional que prefiero
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
# para cambiar su directorio a donde tienen sus files
%cd '/content/drive/My Drive/Colab Notebooks/Curso redes neuronales keras y scikit/red_hat'
%ls

/content/drive/My Drive/Colab Notebooks/Curso redes neuronales keras y scikit/red_hat
act_test.csv  act_train.csv  people.csv  sample_submission.csv


In [5]:
people = pd.read_csv('people.csv', sep = ',', encoding='utf-8')
people.head(3)

people_id  char_1      group_1  char_2  ... char_35 char_36 char_37 char_38
0     ppl_100  type 2  group 17304  type 2  ...    True    True   False      36
1  ppl_100002  type 2   group 8688  type 3  ...    True    True   False      76
2  ppl_100003  type 2  group 33592  type 3  ...   False    True    True      99

[3 rows x 41 columns]

El primer hecho a resaltar es que la información se presenta anonimizada, algo muy común en la ciencia de datos.

Se aprecia una gran cantidad de campos de naturaleza categórica.

In [6]:
#Repetimos el mismo proceso para el csv act_train
activity = pd.read_csv('act_train.csv', sep = ',' )
activity.head(3)

people_id   activity_id        date  ... char_9  char_10 outcome
0   ppl_100  act2_1734928  2023-08-26  ...    NaN  type 76       0
1   ppl_100  act2_2434093  2022-09-27  ...    NaN   type 1       0
2   ppl_100  act2_3404049  2022-09-27  ...    NaN   type 1       0

[3 rows x 15 columns]

# <h1 id="ingenieria">Ingeniería de datos</h1>

Evaluaremos ahora el tamaño de los datasets y la completitud de información en las distintas variables disponibles.

In [7]:
# Verificamos la completitud para el dataset people
print(people.shape)
100*people.isnull().sum()/people.shape[0]

(189118, 41)


people_id    0.0
char_1       0.0
group_1      0.0
char_2       0.0
date         0.0
char_3       0.0
char_4       0.0
char_5       0.0
char_6       0.0
char_7       0.0
char_8       0.0
char_9       0.0
char_10      0.0
char_11      0.0
char_12      0.0
char_13      0.0
char_14      0.0
char_15      0.0
char_16      0.0
char_17      0.0
char_18      0.0
char_19      0.0
char_20      0.0
char_21      0.0
char_22      0.0
char_23      0.0
char_24      0.0
char_25      0.0
char_26      0.0
char_27      0.0
char_28      0.0
char_29      0.0
char_30      0.0
char_31      0.0
char_32      0.0
char_33      0.0
char_34      0.0
char_35      0.0
char_36      0.0
char_37      0.0
char_38      0.0
dtype: float64

In [8]:
#Repetimos el procedimiento con el dataset de activity
print(activity.shape)
100*activity.isnull().sum()/activity.shape[0]

(2197291, 15)


people_id             0.000000
activity_id           0.000000
date                  0.000000
activity_category     0.000000
char_1               92.826849
char_2               92.826849
char_3               92.826849
char_4               92.826849
char_5               92.826849
char_6               92.826849
char_7               92.826849
char_8               92.826849
char_9               92.826849
char_10               7.173151
outcome               0.000000
dtype: float64

Se evidencia que los campos char_1 al char_9 tienen el 92% de sus valores en nulo Esta cantidad es muy elevada para aplicar metodos de imputación, por lo cual optaremos por elminar las columnas con este inconveniente.

In [9]:
activity.columns

Index(['people_id', 'activity_id', 'date', 'activity_category', 'char_1',
       'char_2', 'char_3', 'char_4', 'char_5', 'char_6', 'char_7', 'char_8',
       'char_9', 'char_10', 'outcome'],
      dtype='object')

In [10]:
activity.drop(columns=['char_1','char_2', 'char_3', 'char_4', 'char_5', 'char_6',\
                       'char_7', 'char_8','char_9'],inplace=True)
print(activity.shape)
activity.head(3)

(2197291, 6)


people_id   activity_id        date activity_category  char_10  outcome
0   ppl_100  act2_1734928  2023-08-26            type 4  type 76        0
1   ppl_100  act2_2434093  2022-09-27            type 2   type 1        0
2   ppl_100  act2_3404049  2022-09-27            type 2   type 1        0

In [13]:
# Verificamos nuevamente nuestro dataset
print(activity.shape)
100*activity.isnull().sum()/activity.shape[0]

(2197291, 6)


people_id            0.000000
activity_id          0.000000
date                 0.000000
activity_category    0.000000
char_10              7.173151
outcome              0.000000
dtype: float64

# Consolidando nuestros dataframes en uno solo

Para consolidar un dataset debemos tener un par de consideraciones previas: 

* Se aprecia que existen dos variables que tienen los mismos nombres en ambos 
datasets (char_2 y char_10) por lo cual es buena practica renombrar estas columnas para evitar confusiones al momento de unificar las fuentes.
* En el caso de la variable categórica "tipo_actividad" existe una proporción de nulos que podemos reemplazar por la moda.

In [15]:
#Renombramos columnas
activity = activity.rename(columns={"date":"fecha_actividad","char_10":"tipo_actividad"})

In [16]:
#Reenplazamos el tipo de actividad por la moda 
activity["tipo_actividad"] = activity["tipo_actividad"].fillna(activity["tipo_actividad"].mode()[0])

In [17]:
#Verificamos el cambio
100*activity.isnull().sum()/activity.shape[0]

people_id            0.0
activity_id          0.0
fecha_actividad      0.0
activity_category    0.0
tipo_actividad       0.0
outcome              0.0
dtype: float64

Ahora unificamos los datasets empleando la función merge y evaluamos la consistencia luego de esta operación 

In [18]:
consolidado = activity.merge(people,on=["people_id"],how="inner")
print("Tamaño previo: ",activity.shape)
print("Nuevo tamaño: ",consolidado.shape)


Tamaño previo:  (2197291, 6)
Nuevo tamaño:  (2197291, 46)


La variable objetivo que clasifica la actividad de los usuarios como potenciales clientes es denominada "outcome", evaluaremos la distribución de este variable: 

In [21]:
print("Distribución de outcome: \n", 100*consolidado["outcome"].value_counts()/consolidado.shape[0])

Distribución de outcome: 
 0    55.60456
1    44.39544
Name: outcome, dtype: float64


Como lo hemos mencionado, las redes neuronales requieren de entrada sólo valores numericos, por lo cual debemos convertir aquellas que no lo son. El dataset consolidado cuenta con 46 variables de tipo booleanas, numericas o categóricas que deben ser transformadas.

In [19]:
types = pd.DataFrame(consolidado.dtypes)

In [20]:
print("Tipos de variables",types.groupby(0).size())

Tipos de variables 0
bool      28
int64      2
object    16
dtype: int64


In [22]:
# Hacemos el reemplazo de valores tipo bool hacia int
consolidado = consolidado.replace({False: 0, True: 1})

types = pd.DataFrame(consolidado.dtypes)
print("Tipos de variables luego de reemplazo",types.groupby(0).size())

Tipos de variables luego de reemplazo 0
int64     30
object    16
dtype: int64


In [23]:
print("Distribución de outcome: \n", 100*consolidado["outcome"].value_counts()/consolidado.shape[0])

Distribución de outcome: 
 0    55.60456
1    44.39544
Name: outcome, dtype: float64


# Convirtiendo datos restantes a valores numericos

Si recordamos, el identificador people_id se compone por un prefijo "ppl_" seguido de un número único por usuario.En este caso basta con recortar el prefijo para transformar esta variable en numerica.

In [24]:
consolidado.people_id = consolidado.people_id.str.slice(start=4).astype(float).astype(int)

In [26]:
consolidado.head(3)
#people_id ya es entero

people_id   activity_id fecha_actividad  ... char_36 char_37  char_38
0        100  act2_1734928      2023-08-26  ...       1       0       36
1        100  act2_2434093      2022-09-27  ...       1       0       36
2        100  act2_3404049      2022-09-27  ...       1       0       36

[3 rows x 46 columns]